# Labelstudio Annotator

This notebook is used to add class information based on the contrast of flakes to the annotated flakes from labelstudio.

In [ ]:
import numpy as np
from pycocotools.coco import COCO
from pycocotools.mask import frPyObjects
import os
import cv2
import matplotlib.pyplot as plt
from utils.class_annotator import Class_Annotator
import json

In [ ]:
def create_category_annotation(category_dict):
    category_list = []

    for key, value in category_dict.items():
        category = {"supercategory": key, "id": int(value), "name": key}
        category_list.append(category)

    return category_list


def convert_COCO_to_instance_masks(
    coco_file: COCO,
    file_dir: str = "./",
    mask_dir_name: str = "instance_masks",
) -> np.ndarray:

    os.makedirs(os.path.join(file_dir, mask_dir_name), exist_ok=True)

    coco_imgs = coco_file.loadImgs(coco_file.getImgIds())
    coco_anns = coco_file.loadAnns(coco_file.getAnnIds())

    for coco_img in coco_imgs:
        img_path = os.path.join(file_dir, coco_img["file_name"])
        img_anns = [ann for ann in coco_anns if ann["image_id"] == coco_img["id"]]

        img = cv2.imread(img_path)
        binary_mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
        for i, img_ann in enumerate(img_anns):
            segmentation = [
                np.round(np.array(img_ann["segmentation"]).reshape(-1, 2)).astype(
                    np.int32
                )
            ]
            cv2.fillPoly(binary_mask, np.array(segmentation), i + 1)

        image_name = img_path.split("/")[-1].split("\\")[-1]
        mask_path = os.path.join(
            file_dir, mask_dir_name, image_name.replace(".jpg", ".png")
        )
        cv2.imwrite(
            mask_path,
            binary_mask,
        )


def calculate_background_color(
    img: np.ndarray,
    radius: int = 10,
) -> np.ndarray:
    masks = []

    for i in range(3):
        img_channel = img[:, :, i]
        mask = cv2.inRange(img_channel, 20, 230)
        hist = cv2.calcHist([img_channel], [0], mask, [256], [0, 256])
        hist_mode = np.argmax(hist)
        thresholded_image = cv2.inRange(
            img_channel, int(hist_mode - radius), int(hist_mode + radius)
        )
        background_mask_channel = cv2.erode(
            thresholded_image, np.ones((3, 3)), iterations=3
        )
        masks.append(background_mask_channel)

    final_mask = cv2.bitwise_and(masks[0], masks[1])
    final_mask = cv2.bitwise_and(final_mask, masks[2])

    return np.array(cv2.mean(img, mask=final_mask)[:3])


def get_instance_contrasts_from_coco_file(
    coco_file: COCO,
    file_dir: str,
    min_instance_size: int = 200,
) -> tuple[list[np.ndarray], list[dict]]:
    # returns a list of all instances
    # each instance is a N x 3 Array with N being the number of pixels in the instance and 3 being the BGR values
    instance_contrasts = []
    instance_classifiers = []

    coco_imgs = coco_file.loadImgs(coco_file.getImgIds())
    coco_anns = coco_file.loadAnns(coco_file.getAnnIds())

    for coco_img in coco_imgs:
        img_path = os.path.join(file_dir, coco_img["file_name"])
        img_anns = [ann for ann in coco_anns if ann["image_id"] == coco_img["id"]]

        img = cv2.imread(img_path)
        background_color = calculate_background_color(img)

        if np.any(background_color == 0):
            print(f"Error with image {img_path}; Invalid Background, skipping")
            continue

        contrast_img = img / background_color - 1

        for i, img_ann in enumerate(img_anns):
            segmentation = [
                np.round(np.array(img_ann["segmentation"]).reshape(-1, 2)).astype(
                    np.int32
                )
            ]

            binary_mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
            cv2.fillPoly(binary_mask, np.array(segmentation), 1)

            if cv2.countNonZero(binary_mask) < min_instance_size:
                continue

            cv2.erode(binary_mask, np.ones((3, 3)), iterations=3)

            instance_contrasts.append(contrast_img[binary_mask == 1])
            instance_classifiers.append(img_ann)

    return instance_contrasts, instance_classifiers


def polygon_seg_to_rle_seg(in_path, out_path):
    with open(in_path, "r") as f:
        raw_coco_file = json.load(f)

    images = raw_coco_file["images"]
    anns = raw_coco_file["annotations"]

    for image in images:
        for ann in anns:
            if ann["image_id"] == image["id"]:
                ann["segmentation"] = frPyObjects(
                    ann["segmentation"], image["height"], image["width"]
                )[0]
                ann["segmentation"]["counts"] = ann["segmentation"]["counts"].decode(
                    "utf-8"
                )

    with open(out_path, "w") as f:
        json.dump(raw_coco_file, f, indent=4)

In [ ]:
FILE_DIR = "Path/to/the/labelstudio/dataset"
ANNOTATION_FILE = os.path.join(FILE_DIR, "result.json")
RLE_ANNOTATION_FILE = os.path.join(FILE_DIR, "RLE_result.json")
OUT_ANNOTATION_FILE = os.path.join(FILE_DIR, "result_with_class.json")
RLE_OUT_ANNOTATION_FILE = os.path.join(FILE_DIR, "RLE_result_with_class.json")

polygon_seg_to_rle_seg(ANNOTATION_FILE, RLE_ANNOTATION_FILE)
coco_file = COCO(ANNOTATION_FILE)

In [ ]:
instance_contrasts, instance_classifiers = get_instance_contrasts_from_coco_file(
    coco_file,
    FILE_DIR,
    min_instance_size=200,
)

In [ ]:
%matplotlib qt

CA = Class_Annotator(
    instance_contrasts,
    plot_alpha=1,
    plot_s=50,
    display_std=True,
)

CA.run()

In [ ]:
instance_classes = CA.get_results()
unique_instance_classes = np.unique(instance_classes).astype(int)
class_contrasts = {cls_id: [] for cls_id in unique_instance_classes}

for instance_contrast, class_id in zip(instance_contrasts, instance_classes):
    class_contrasts[class_id].extend(instance_contrast)

for class_id, contrasts in class_contrasts.items():
    class_contrasts[class_id] = np.array(contrasts)
    print(f"Class {class_id} has {len(contrasts)} datapoints")

new_instance_classifiers = []
for instance_class, instance_classifier in zip(instance_classes, instance_classifiers):
    if instance_class == 0:
        continue
    new_instance_classifier = instance_classifier.copy()
    new_instance_classifier["category_id"] = int(instance_class)
    new_instance_classifiers.append(new_instance_classifier)

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(1, 1, figsize=(5, 5))

for class_id, contrasts in class_contrasts.items():
    if class_id == 0:
        continue
    ax.scatter(
        contrasts[:, 0],
        contrasts[:, 2],
        label=f"Class {class_id}",
        alpha=1 / 255,
        ec="none",
    )

ax.legend()

In [ ]:
with open(ANNOTATION_FILE, "r") as f:
    raw_coco_file = json.load(f)

raw_coco_file["categories"] = create_category_annotation(
    {f"Class {class_id}": class_id for class_id in unique_instance_classes}
)

raw_coco_file["annotations"] = new_instance_classifiers

with open(OUT_ANNOTATION_FILE, "w") as f:
    json.dump(raw_coco_file, f, indent=4)

polygon_seg_to_rle_seg(OUT_ANNOTATION_FILE, RLE_OUT_ANNOTATION_FILE)